In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv("C:\\Users\\DELL\\Downloads\\finanical_information.csv")
df2 = pd.read_csv("C:\\Users\\DELL\\Downloads\\industry_client_details.csv")
df3 = pd.read_csv("C:\\Users\\DELL\\Downloads\\payment_information.csv")
df4 = pd.read_csv("C:\\Users\\DELL\\Downloads\\subscription_information.csv") #To run this notebook again, need to change the path with corresponding file path in local system

In [3]:
df1.head() # Check the data of the tables

,Unnamed: 0,start_date,end_date,inflation_rate,gdp_growth_rate
0,0,2018-01-01,2018-03-31,5.77,3.51
1,1,2018-04-01,2018-06-30,1.17,2.15
2,2,2018-07-01,2018-09-30,1.56,1.82
3,3,2018-10-01,2018-12-31,2.78,2.43
4,4,2019-01-01,2019-03-31,6.91,3.44


In [4]:
df2.head()

,client_id,company_size,industry,location
0,4280387012,Large,Finance Lending,Mumbai
1,2095513148,Small,Finance Lending,Chennai
2,7225516707,Medium,Finance Lending,New Delhi
3,8093537819,Large,Block Chain,Mumbai
4,4387541014,Medium,Hyper Local,Banglore


In [5]:
df3.head()

,client_id,payment_date,amount_paid,payment_method
0,6292156167,9/16/2019,447.0,Bank Transfer
1,7462725203,5/21/2018,379.7,Bank Transfer
2,4698004907,9/11/2021,435.1,Check
3,3510240337,12/7/2020,413.1,Check
4,7501599785,3/4/2019,61.1,Bank Transfer


In [6]:
df4.head()

,client_id,subscription_type,start_date,end_date,renewed
0,1131383004,Yearly,2020-11-11,2021-11-11,False
1,4309371709,Monthly,2021-05-24,2021-06-23,True
2,3183675157,Yearly,2021-12-25,2022-12-25,True
3,5371694837,Monthly,2020-03-14,2020-04-13,True
4,5157113076,Monthly,2019-11-07,2019-12-07,False


In [7]:
print(df1.isna().sum())  # Checking for any empty values in all the tables

Unnamed: 0         0
start_date         0
end_date           0
inflation_rate     0
gdp_growth_rate    0
dtype: int64


In [8]:
print(df2.isna().sum())

client_id       0
company_size    0
industry        0
location        0
dtype: int64


In [9]:
print(df3.isna().sum())

client_id         0
payment_date      0
amount_paid       0
payment_method    0
dtype: int64


In [10]:
print(df4.isna().sum())

client_id            0
subscription_type    0
start_date           0
end_date             0
renewed              0
dtype: int64


In [12]:
df2['industry'].value_counts() # Get the no of values for the column industry to get the count of the different industry clients

Block Chain        25
Finance Lending    22
Gaming             22
Hyper Local        20
AI                 11
Name: industry, dtype: int64

1.As Per the above result, We can observe that there are 25 block chain clients and 22 finance lending clients for the organization

In [13]:
renewed_clients = df4[df4['renewed'] == True] # get the clients who are renewing their subscription

In [14]:
renewed_clients_information = pd.merge(renewed_clients,df2,how='inner',on='client_id') # Join to the clients info dataframe

In [15]:
renewed_clients_information.head() # checking the table 

,client_id,subscription_type,start_date,end_date,renewed,company_size,industry,location
0,4309371709,Monthly,2021-05-24,2021-06-23,True,Large,Hyper Local,Hyderabad
1,3183675157,Yearly,2021-12-25,2022-12-25,True,Small,Gaming,Chennai
2,5371694837,Monthly,2020-03-14,2020-04-13,True,Large,AI,New Delhi
3,7896208406,Yearly,2022-02-24,2023-02-24,True,Small,Block Chain,New Delhi
4,4687291312,Yearly,2019-06-14,2020-06-13,True,Medium,Block Chain,New Delhi


In [16]:
industry_wise_renewed_count = renewed_clients_information.pivot_table(values='renewed',index='industry',aggfunc='count') #Creating a pivot table for industry wise renewal

In [17]:
industry_wise_renewed_count['renewal_rate'] = round((industry_wise_renewed_count['renewed']/industry_wise_renewed_count['renewed'].sum())*100,2)

In [18]:
print(industry_wise_renewed_count['renewal_rate'].idxmax()) # Calculating the renewal rate and get the industry which has the max renewal rate.

Gaming


2. As per the above result, We can conclude that gaming industry has the highest renewal rate

In [19]:
def get_quarter(date):     #Creating function to get quarter of the year
    date=pd.to_datetime(date)
    year=date.year
    quarter = date.quarter
    return f"{year}-Q{quarter}"
    

In [20]:
client_payment_info = pd.merge(df3,df4,on='client_id',how='inner')[['client_id','payment_date','renewed']] # join payments and subscriptions table 

In [21]:
client_payment_info['quarter'] = client_payment_info['payment_date'].apply(get_quarter) # create new quarter column for merged dataframe

In [22]:
df1['quarter'] = df1['start_date'].apply(get_quarter) # create quarter column for financial information

In [23]:
inflation_rate_client_subscription = pd.merge(client_payment_info,df1,on='quarter',how='inner')[['client_id','payment_date','inflation_rate','renewed','quarter']]

In [24]:
inflation_rate_client_subscription_renewed = inflation_rate_client_subscription[inflation_rate_client_subscription['renewed'] == True]

In [25]:
average_inflation_rate = round(inflation_rate_client_subscription_renewed['inflation_rate'].mean(),2)
print(average_inflation_rate)  #get only renewed subscriptions and calculation average of the inflation rate

4.44


3. From the result, When the payments for the subscription are made the average inflation rate is 4.44

In [26]:
df3['payment_date'] = pd.to_datetime(df3['payment_date']) # convert payment date to datetime datatype

In [27]:
df3['year'] = df3['payment_date'].dt.year # Extract the year of date and create new column

In [28]:
payment_info_stats = df3.pivot_table(values='amount_paid',index='year',columns='payment_method',aggfunc='median') # create a new stats table 

In [29]:
payment_info_stats # Final result 

payment_method,Bank Transfer,Check,Credit Card
year,,,
2018,281.65,216.6,229.15
2019,184.20,410.2,401.90
2020,225.10,413.1,285.25
2021,255.30,435.1,208.70
2022,196.50,275.5,326.20


4.Above result represents the each year median amount received for each payment method